## Collect and preprocess data

# 1. Set-up environment

In [1]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import xmltodict
import re
import random
import os

# 2. Collect data from a website 

**1. Trang web tiến hành thu thập:** https://data.gov/ là trang web dữ liệu mở của chính phủ Hoa Kỳ. Nó cung cấp quyền truy cập vào các bộ dữ liệu được xuất bản bởi các cơ quan trong chính phủ liên bang. Trang web nhằm mục đích cung cấp quyền truy cập vào dữ liệu mở của chính phủ cho công chúng, đạt được các sứ mệnh của cơ quan, thúc đẩy đổi mới, thúc đẩy hoạt động kinh tế và duy trì lý tưởng về một chính phủ cởi mở và minh bạch.

**2. Chủ đề thực hiện: Crimes**
- Crimes - 2001 to Present: Tập dữ liệu này phản ánh các vụ tội phạm được báo cáo (ngoại trừ các vụ giết người có dữ liệu về từng nạn nhân) xảy ra ở Thành phố Chicago từ năm 2001 đến nay. Dữ liệu được trích xuất từ hệ thống CLEAR (Phân tích và Báo cáo Thực thi Pháp luật Công dân) của Sở Cảnh sát Chicago.
- Link: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/data_preview

- Do tập dữ liệu lớn nếu lấy từ 2001 tới 2023 là rất lớn nên nhóm chọn ra 3 năm gần đây để thực hiện.
- Với năm 2023 lúc tiến hành chưa kết thúc nên dữ liệu sẽ bị thiếu nhiều vào tháng 12/2023 nên chọn năm 2020 2021 và 2022.
- URL năm 2020: https://data.cityofchicago.org/Public-Safety/Crimes-2020/qzdf-xmn8/data_preview
- URL năm 2021: https://data.cityofchicago.org/Public-Safety/Crimes-2021/dwme-t96c/data_preview
- URL năm 2022: https://data.cityofchicago.org/Public-Safety/Crimes-2022/9hwr-2zxp/data_preview

**Dưới đây là bảng dữ liệu với các thuộc tính tương ứng:**

- `ID`: Mã định danh.
- `Case Number`: Mã số vụ án.
- `Date`: Ngày và giờ của vụ án.
- `Block`: Địa chỉ nơi sự cố xảy ra.
- `IUCR`: Mã báo cáo loại tội phạm đồng nhất Illinois.
- `Primary Type`: Mô tả chính về loại tội phạm.
- `Description`: Mô tả cụ thể về tội phạm.
- `Location Description`: Mô tả cụ thể nơi xảy ra sự cố.
- `Arrest`: Cho biết liệu có bắt giữ nghi phạm hay không (Đúng/Sai).
- `Domestic`: Cho biết liệu vụ việc có liên quan đến vấn đề nội tâm gia đình hay không (Đúng/Sai).
- `Beat`: Khu vực quản lý của cảnh sát nơi sự cố xảy ra.
- `District`: Mã quận cảnh sát nơi sự cố xảy ra.
- `Ward`: Mã phường xảy ra sự cố.
- `Community Area`: Mã khu cộng đồng xảy ra sự cố.
- `FBI Code`: Mã từ hệ thống phân loại tội phạm của FBI.
- `X Coordinate`: Tọa độ x của vị trí xảy ra sự cố trong phép chiếu State Plane Illinois East NAD 1983
- `Y Coordinate`: Tọa độ y của vị trí xảy ra sự cố trong phép chiếu State Plane Illinois East NAD 1983
- `Year`: Năm của sự kiện.
- `Updated On`: Ngày và giờ khi bản ghi được cập nhật lần cuối.
- `Latitude`: Vĩ độ của địa điểm xảy ra sự cố
- `Longitude`: Kinh độ của địa điểm xảy ra sự cố
- `Location`: Vị trí xảy ra sự cố ở định dạng cho phép tạo bản đồ và các hoạt động địa lý khác trên cổng dữ liệu.

Để bảo vệ quyền riêng tư của nạn nhân tội phạm, địa chỉ chỉ được hiển thị ở cấp khối và các địa điểm cụ thể không được xác định.

In [2]:
url1 = 'https://data.cityofchicago.org/Public-Safety/Crimes-2020/qzdf-xmn8/data_preview'
url2 = 'https://data.cityofchicago.org/Public-Safety/Crimes-2021/dwme-t96c/data_preview'
url3 = 'https://data.cityofchicago.org/Public-Safety/Crimes-2022/9hwr-2zxp/data_preview'
response1 = requests.get(url1)
response2 = requests.get(url2)
response3 = requests.get(url3)
html_content_raw = response1.content + response2.content + response3.content

### Lọc dữ liệu chọn 3000 dòng từ 3 trang web

In [3]:
values = list(range(200001))
random_values = random.sample(values, 1000)
sorted_values = sorted(random_values) 
string_values = [str(value) for value in sorted_values]  
result_string = ', '.join(string_values)
rows_index = result_string

soup = BeautifulSoup(html_content_raw, 'html.parser')
target_rows = soup.find_all('div', {'row-index': rows_index})
soup = str(target_rows)

### Viết hàm Crawl_col để truy xuất vào phần tử HTML

In [4]:
def Crawl_Col(target_col):
    attribute = []
    target_cells = soup.find_all('div', {'col-id': target_col})
    if target_cells:
        for data in target_cells:
            data = data.find('div').text.strip()
            attribute.append(data)
    else:
        attribute.append('NaN')

    return attribute

### Tiến hành thu thập dữ liệu

In [5]:

ID = []
Case_Number = []
Block = []
Date = []
IUCR = []
Primary_Type = []
Description = []
Location_Description = []
Arrest = []
Domestic = []
Beat = []
District = []
Ward = []
Community_Area = []
FBI_Code = []
X_Coordinate = []
Y_Coordinate = []
Year = []
Updated_On = []
Latitude = []
Longitude = []
Location = []
    
ID = Crawl_Col('id')
Case_Number = Crawl_Col('case_number')
Block = Crawl_Col('block')
Date = Crawl_Col('date')
IUCR = Crawl_Col('iucr')
Primary_Type = Crawl_Col('primary_type')
Description = Crawl_Col('description')
Location_Description = Crawl_Col('location_description')
Arrest = Crawl_Col('arrest')
Domestic = Crawl_Col('domestic')
Beat = Crawl_Col('beat')
District = Crawl_Col('district')
Ward = Crawl_Col('ward')
Community_Area = Crawl_Col('community_area')
FBI_Code = Crawl_Col('fbi_code')
X_Coordinate = Crawl_Col('x_coordinate')
Y_Coordinate = Crawl_Col('y_coordinate')
Year = Crawl_Col('year')
Updated_On = Crawl_Col('updated_on')
Latitude = Crawl_Col('latitude')
Longitude = Crawl_Col('longitude')
Location = Crawl_Col('location')
    
data = pd.DataFrame({"ID": ID,
                     "Case Number": Case_Number,
                     "Date": Date,
                     "Block": Block,
                     "IUCR": IUCR,
                     "Primary Type": Primary_Type,
                     "Description": Description,
                     "Location Description": Location_Description,
                     "Arrest": Arrest,
                     "Domestic": Domestic,
                     "Beat": Beat,
                     "District": District,
                     "Ward": Ward,
                     "Community Area": Community_Area,
                     "FBI Code": FBI_Code,
                     "X Coordinate": X_Coordinate,
                     "Y Coordinate": Y_Coordinate,
                     "Year": Year,
                     "Updated On": Updated_On,
                     "Latitude": Latitude,
                     "Longitude": Longitude,
                     "Location": Location})
data

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12329132,JE185820,03/31/2021 07:00:00 PM,033XX N PULASKI RD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,...,30.0,21.0,26,1149137.0,1921951.0,2021.0,04/07/2021 03:40:57 PM,41.941762,-87.727267,"(41.941762494, -87.727267038)"
1,12813034,JF379127,09/01/2022 07:00:00 AM,007XX W WAYMAN ST,0820,THEFT,$500 AND UNDER,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,42.0,28.0,06,1171358.0,1902334.0,2022.0,01/03/2023 03:40:27 PM,41.887472,-87.646174,"(41.887472275, -87.646173514)"
2,12912192,JF497936,12/04/2022 03:00:00 PM,106XX S RIDGEWAY AVE,0460,BATTERY,SIMPLE,DRIVEWAY - RESIDENTIAL,False,False,...,19.0,74.0,08B,1153233.0,1833736.0,2022.0,01/03/2023 03:40:27 PM,41.699608,-87.714547,"(41.699608315, -87.714547408)"
3,12690514,JF231805,05/05/2022 12:50:00 PM,081XX S COTTAGE GROVE AVE,0560,ASSAULT,SIMPLE,RESIDENCE,False,False,...,8.0,44.0,08A,1182972.0,1851219.0,2022.0,01/03/2023 03:40:27 PM,41.746946,-87.605115,"(41.746945834, -87.605114755)"
4,12555760,JE462959,12/01/2021 09:20:00 PM,003XX S WHIPPLE ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,STREET,False,False,...,28.0,27.0,11,1156097.0,1898367.0,2021.0,12/08/2021 03:44:18 PM,41.876908,-87.702324,"(41.876908113, -87.702323689)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,12449440,JE333209,08/10/2021 08:00:00 PM,056XX W IOWA ST,0560,ASSAULT,SIMPLE,APARTMENT,False,False,...,29.0,25.0,08A,1138423.0,1905439.0,2021.0,08/17/2021 03:41:37 PM,41.896653,-87.767047,"(41.89665282, -87.767046737)"
2996,12687960,JF228706,05/03/2022 10:00:00 AM,087XX S ASHLAND AVE,0560,ASSAULT,SIMPLE,COMMERCIAL / BUSINESS OFFICE,False,False,...,21.0,71.0,08A,1167194.0,1846860.0,2022.0,01/03/2023 03:40:27 PM,41.735336,-87.663054,"(41.735335992, -87.663053811)"
2997,12025738,JD204255,04/07/2020 06:45:00 PM,061XX S COTTAGE GROVE AVE,051A,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,True,...,20.0,42.0,04A,1182617.0,1864574.0,2020.0,09/07/2021 03:41:02 PM,41.783602,-87.606002,"(41.783601519, -87.606001759)"
2998,12582627,JF100933,01/01/2022 11:46:00 PM,038XX W ARTHINGTON ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,24.0,26.0,08B,1150855.0,1895759.0,2022.0,01/03/2023 03:40:27 PM,41.869856,-87.721639,"(41.86985561, -87.721639097)"


### Lưu data vào file csv

In [6]:
if not os.path.exists('data'):
    os.makedirs('data')
file_path = os.path.join('data', 'Crimes.csv')

data.to_csv(file_path, index=False)